In [49]:
%matplotlib inline

from __future__ import division

import dfmaker
import modelmaker
import AnaFunc

reload(dfmaker)
reload(modelmaker)
reload(AnaFunc)

import time

from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

import numpy as np

import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn import metrics
from sklearn.utils import shuffle

from collections import defaultdict

In [94]:
def AuthorStats(tag,myconfig):
    df_all = dfmaker.get_train_dfs(tag,myconfig)
    df_reg = df_all[df_all['evtclass']==0]
    df_spo = df_all[df_all['evtclass']==1]
    unique_reg = df_reg['blogname'].value_counts()
    unique_spo = df_spo['blogname'].value_counts()

    spolen = df_spo.shape[0]
    print "Spoilers: %d posts" % spolen
    print "  Mean: %f Median: %d Max: %d" % (unique_spo.mean(), unique_spo.median(), unique_spo.max())
    print "  Mean: %f%% Median: %f%% Max: %f%%" % (100*unique_spo.mean()/spolen, \
                                            100*unique_spo.median()/spolen, \
                                            100*unique_spo.max()/spolen)
    reglen = df_reg.shape[0]
    print "Spoilers: %d posts" % reglen
    print "  Mean: %f Median: %d Max: %d" % (unique_reg.mean(), unique_reg.median(), unique_reg.max())
    print "  Mean: %f%% Median: %f%% Max: %f%%" % (100*unique_reg.mean()/reglen, \
                                            100*unique_reg.median()/reglen, \
                                            100*unique_reg.max()/reglen)
    
    post_limit = int(0.0005*(df_all.shape[0]))
    if post_limit == 0:
        post_limit = 1
        
    print "LIMIT IS:",post_limit
    
    auth_select = np.empty(df_all.shape[0],dtype=bool)
    
    df_all = shuffle(df_all,random_state=20)
    auth_count = defaultdict(int)
    
    for i in range(0, df_all.shape[0]):
        bname = df_all.iloc[i]['blogname']
        auth_count[bname] += 1
        if auth_count[bname]>post_limit:
            auth_select[i] = False
        else :
            auth_select[i] = True

    df_all = df_all[auth_select]
    
    print "NEXT ROUND"
    
    df_reg = df_all[df_all['evtclass']==0]
    df_spo = df_all[df_all['evtclass']==1]
    unique_reg = df_reg['blogname'].value_counts()
    unique_spo = df_spo['blogname'].value_counts()

    spolen = df_spo.shape[0]
    print "Spoilers: %d posts" % spolen
    print "  Mean: %f Median: %d Max: %d" % (unique_spo.mean(), unique_spo.median(), unique_spo.max())
    print "  Mean: %f%% Median: %f%% Max: %f%%" % (100*unique_spo.mean()/spolen, \
                                            100*unique_spo.median()/spolen, \
                                            100*unique_spo.max()/spolen)
    reglen = df_reg.shape[0]
    print "Regular: %d posts" % reglen
    print "  Mean: %f Median: %d Max: %d" % (unique_reg.mean(), unique_reg.median(), unique_reg.max())
    print "  Mean: %f%% Median: %f%% Max: %f%%" % (100*unique_reg.mean()/reglen, \
                                            100*unique_reg.median()/reglen, \
                                            100*unique_reg.max()/reglen)

    return df_all

In [95]:
myconfig = "/Users/ruthtoner/CodingMacros/ProjectInsight/myconfigs.cfg"


#Vectorizer:
vect_text = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 500, \
                             max_df = 0.8)

vect_tag = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 200, \
                             max_df = 0.8, \
                             binary = True)
#Random Forest:
forest = RandomForestClassifier(n_estimators = 100)

In [96]:
df_all_reduced = AuthorStats("sw",myconfig)

Spoilers: 12626 posts
  Mean: 1.684590 Median: 1 Max: 308
  Mean: 0.013342% Median: 0.007920% Max: 2.439411%
Spoilers: 30096 posts
  Mean: 1.309661 Median: 1 Max: 233
  Mean: 0.004352% Median: 0.003323% Max: 0.774189%
LIMIT IS: 21
NEXT ROUND
Spoilers: 12009 posts
  Mean: 1.602268 Median: 1 Max: 21
  Mean: 0.013342% Median: 0.008327% Max: 0.174869%
Regular: 29422 posts
  Mean: 1.280442 Median: 1 Max: 21
  Mean: 0.004352% Median: 0.003399% Max: 0.071375%


In [97]:
modelmaker.model_cv(df_all_reduced,forest,vect_text,vect_tag,n_folds=5,downsample=True)

Mean AUC = 0.850661, Std = 0.003860


In [98]:
df_all_reduced = AuthorStats("dai",myconfig)

Spoilers: 19331 posts
  Mean: 6.136825 Median: 2 Max: 232
  Mean: 0.031746% Median: 0.010346% Max: 1.200145%
Spoilers: 20653 posts
  Mean: 2.994490 Median: 1 Max: 188
  Mean: 0.014499% Median: 0.004842% Max: 0.910279%
LIMIT IS: 19
NEXT ROUND
Spoilers: 12283 posts
  Mean: 3.909293 Median: 2 Max: 19
  Mean: 0.031827% Median: 0.016283% Max: 0.154685%
Regular: 17206 posts
  Mean: 2.501963 Median: 1 Max: 19
  Mean: 0.014541% Median: 0.005812% Max: 0.110427%


In [99]:
modelmaker.model_cv(df_all_reduced,forest,vect_text,vect_tag,n_folds=5,downsample=True)

Mean AUC = 0.804705, Std = 0.004244


In [100]:
df_all_reduced = AuthorStats("aou",myconfig)

Spoilers: 4795 posts
  Mean: 1.597801 Median: 1 Max: 49
  Mean: 0.033322% Median: 0.020855% Max: 1.021898%
Spoilers: 9428 posts
  Mean: 1.215889 Median: 1 Max: 31
  Mean: 0.012897% Median: 0.010607% Max: 0.328808%
LIMIT IS: 7
NEXT ROUND
Spoilers: 4443 posts
  Mean: 1.481494 Median: 1 Max: 7
  Mean: 0.033344% Median: 0.022507% Max: 0.157551%
Regular: 9285 posts
  Mean: 1.198065 Median: 1 Max: 7
  Mean: 0.012903% Median: 0.010770% Max: 0.075390%


In [101]:
modelmaker.model_cv(df_all_reduced,forest,vect_text,vect_tag,n_folds=5,downsample=True)

Mean AUC = 0.862395, Std = 0.007524


In [102]:
df_all_reduced = AuthorStats("madmax",myconfig)

Spoilers: 334 posts
  Mean: 1.197133 Median: 1 Max: 4
  Mean: 0.358423% Median: 0.299401% Max: 1.197605%
Spoilers: 1361 posts
  Mean: 1.102024 Median: 1 Max: 5
  Mean: 0.080972% Median: 0.073475% Max: 0.367377%
LIMIT IS: 1
NEXT ROUND
Spoilers: 271 posts
  Mean: 1.000000 Median: 1 Max: 1
  Mean: 0.369004% Median: 0.369004% Max: 0.369004%
Regular: 1219 posts
  Mean: 1.000000 Median: 1 Max: 1
  Mean: 0.082034% Median: 0.082034% Max: 0.082034%


In [103]:
modelmaker.model_cv(df_all_reduced,forest,vect_text,vect_tag,n_folds=5,downsample=True)

Mean AUC = 0.835532, Std = 0.018529


In [104]:
df_all_reduced = AuthorStats("utale",myconfig)

Spoilers: 18217 posts
  Mean: 2.525229 Median: 1 Max: 309
  Mean: 0.013862% Median: 0.005489% Max: 1.696218%
Spoilers: 31587 posts
  Mean: 1.575176 Median: 1 Max: 274
  Mean: 0.004987% Median: 0.003166% Max: 0.867445%
LIMIT IS: 24
NEXT ROUND
Spoilers: 16152 posts
  Mean: 2.239290 Median: 1 Max: 24
  Mean: 0.013864% Median: 0.006191% Max: 0.148588%
Regular: 30851 posts
  Mean: 1.539087 Median: 1 Max: 24
  Mean: 0.004989% Median: 0.003241% Max: 0.077793%


In [105]:
modelmaker.model_cv(df_all_reduced,forest,vect_text,vect_tag,n_folds=5,downsample=True)

Mean AUC = 0.793292, Std = 0.001886


In [106]:
df_all_reduced = AuthorStats("lis",myconfig)

Spoilers: 9960 posts
  Mean: 2.483171 Median: 1 Max: 130
  Mean: 0.024931% Median: 0.010040% Max: 1.305221%
Spoilers: 13000 posts
  Mean: 1.692708 Median: 1 Max: 131
  Mean: 0.013021% Median: 0.007692% Max: 1.007692%
LIMIT IS: 11
NEXT ROUND
Spoilers: 8391 posts
  Mean: 2.093041 Median: 1 Max: 11
  Mean: 0.024944% Median: 0.011918% Max: 0.131093%
Regular: 12230 posts
  Mean: 1.594108 Median: 1 Max: 11
  Mean: 0.013034% Median: 0.008177% Max: 0.089943%


In [107]:
modelmaker.model_cv(df_all_reduced,forest,vect_text,vect_tag,n_folds=5,downsample=True)

Mean AUC = 0.791915, Std = 0.005853


In [108]:
df_all_reduced = AuthorStats("slock",myconfig)

Spoilers: 17408 posts
  Mean: 2.076336 Median: 1 Max: 233
  Mean: 0.011927% Median: 0.005744% Max: 1.338465%
Spoilers: 23596 posts
  Mean: 1.441593 Median: 1 Max: 264
  Mean: 0.006109% Median: 0.004238% Max: 1.118834%
LIMIT IS: 20
NEXT ROUND
Spoilers: 16374 posts
  Mean: 1.953006 Median: 1 Max: 20
  Mean: 0.011927% Median: 0.006107% Max: 0.122145%
Regular: 23070 posts
  Mean: 1.409888 Median: 1 Max: 20
  Mean: 0.006111% Median: 0.004335% Max: 0.086693%


In [109]:
modelmaker.model_cv(df_all_reduced,forest,vect_text,vect_tag,n_folds=5,downsample=True)

Mean AUC = 0.738472, Std = 0.003493


In [110]:
df_all_reduced = AuthorStats("han",myconfig)

Spoilers: 18447 posts
  Mean: 3.895882 Median: 1 Max: 221
  Mean: 0.021119% Median: 0.005421% Max: 1.198027%
Spoilers: 25513 posts
  Mean: 2.440501 Median: 1 Max: 246
  Mean: 0.009566% Median: 0.003920% Max: 0.964214%
LIMIT IS: 21
NEXT ROUND
Spoilers: 13974 posts
  Mean: 2.951838 Median: 1 Max: 21
  Mean: 0.021124% Median: 0.007156% Max: 0.150279%
Regular: 21991 posts
  Mean: 2.105409 Median: 1 Max: 21
  Mean: 0.009574% Median: 0.004547% Max: 0.095494%


In [111]:
modelmaker.model_cv(df_all_reduced,forest,vect_text,vect_tag,n_folds=5,downsample=True)

Mean AUC = 0.709493, Std = 0.005843


In [112]:
df_all_reduced = AuthorStats("got",myconfig)

Spoilers: 16419 posts
  Mean: 2.116669 Median: 1 Max: 318
  Mean: 0.012892% Median: 0.006091% Max: 1.936781%
Spoilers: 24350 posts
  Mean: 1.444932 Median: 1 Max: 217
  Mean: 0.005934% Median: 0.004107% Max: 0.891170%
LIMIT IS: 20
NEXT ROUND
Spoilers: 14965 posts
  Mean: 1.929474 Median: 1 Max: 20
  Mean: 0.012893% Median: 0.006682% Max: 0.133645%
Regular: 23281 posts
  Mean: 1.382154 Median: 1 Max: 20
  Mean: 0.005937% Median: 0.004295% Max: 0.085907%


In [113]:
modelmaker.model_cv(df_all_reduced,forest,vect_text,vect_tag,n_folds=5,downsample=True)

Mean AUC = 0.770688, Std = 0.007380
